In [1]:
import pandas as pd
df = pd.read_csv("Banking_Database.csv")

In [2]:
from langchain.docstore.document import Document
docs = []
for _, row in df.iterrows():
    text = "\n".join([f"{col}: {row[col]}" for col in df.columns])
    docs.append(Document(page_content=text))

In [4]:
from transformers import pipeline, AutoTokenizer, AutoModelForSeq2SeqLM
from langchain.llms import HuggingFacePipeline

model_name = "google/flan-t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

pipe = pipeline("text2text-generation", model=model, tokenizer=tokenizer, max_length=256)

llm = HuggingFacePipeline(pipeline=pipe)


Device set to use cpu
C:\Users\hp\AppData\Local\Temp\ipykernel_10076\1577249783.py:11: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)


In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceBgeEmbeddings
from langchain.chains import RetrievalQA
from langchain.docstore.document import Document
import faiss  

embedding_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5") 
db = FAISS.from_documents(docs, embedding_model)

C:\Users\hp\AppData\Local\Temp\ipykernel_10076\1771090114.py:7: LangChainDeprecationWarning: The class `HuggingFaceBgeEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceBgeEmbeddings(model_name="BAAI/bge-small-en-v1.5")
c:\Users\hp\Desktop\Last try\vetest\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\hp\.cache\huggingface\hub\models--BAAI--bge-small-en-v1.5. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE

In [6]:
qa_chain = RetrievalQA.from_chain_type(
    llm = llm,
    retriever = db.as_retriever()
)

In [7]:
query = "Who are the customers that have more than one credit card?"
answer = qa_chain.run(query)
print("Answer:", answer)

C:\Users\hp\AppData\Local\Temp\ipykernel_10076\605789472.py:2: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run(query)
Token indices sequence length is longer than the specified maximum sequence length for this model (1107 > 512). Running this sequence through the model will result in indexing errors


Answer: Betty Clark, Amanda Gonzalez, Margaret Hill, and Perez
